# Get 17 raw features

In this script we extracts 17 raw features from datafiles we generated in 8_processing.ipynb.

After this step, we have the following output files:
- DB_merged_Xhrs.npy: matrices of time series of each admission. For each admission, there is a matrix containing its records in given time period. Each row of the matrix is like this: [feature 0, …, feature n, number of seconds in [icu intime, current time ], admission_id].
- ICD9-Xhrs.npy: matrices of ICD9 codes of each admission. For each admission, there is a matrix containing its ICD9 codes. Each line of the matrix is like this: [admission_id, icd9 original code, icd9 parsed code, icd9 subcat number]
- AGE_LOS_MORTALITY_Xhrs.npy: matrices of the result of AGE_LOS_MORTALITY function for each admission. Here we just keep it for compatibility.
- ADM_FEATURES_Xhrs.npy: features only related to admissions and not related to time, containing age, whether there is AIDS/hematologic malignancy/metastatic cancer and admission type.
- ADM_LABELS_Xhrs.npy: mortality labels of all admissions, containing hospital mortality, 1/2/3-day mortality, 30-day mortality and 1-year mortality.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import shutil
from collections import OrderedDict

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

%matplotlib inline

# First to extract 17 features

In [2]:
SOURCEDIR = '../../Data/admdata_valid/'
TARGETDIR = '../../Data/admdata_17f/'
LABELDIR = './admdata_timelabels/'
RAWDIR = os.path.join(TARGETDIR, 'raw')
PROCESSED_DB_DIR = os.path.join(TARGETDIR, 'processed_db')

if not os.path.exists(TARGETDIR):
    os.makedirs(TARGETDIR)
if not os.path.exists(RAWDIR):
    os.makedirs(RAWDIR)
    
valid_aids = [re.match(r'adm\-(\d+)\.npy', x) for x in os.listdir(SOURCEDIR)]
valid_aids = sorted([int(x.group(1)) for x in valid_aids if x is not None])
print(len(valid_aids), valid_aids[:10])

map_itemid_index = np.load('res/map_itemid_index.npy').tolist()

38425 [100001, 100006, 100007, 100009, 100010, 100011, 100012, 100016, 100017, 100021]


In [3]:
# merge selected items to one item; the value is the mean of all values
def keep_nonneg(x):
    try:
        x = float(x)
        return x >= 0
    except:
        return True

def merge_items(timeseries, mergeset, new_column_id, merge_func=np.mean, keep_func=keep_nonneg):
    merged = {}
    for record in timeseries:
        i, j, value = record[0], record[1], record[2]
        if j in mergeset:
#             try:
#                 value = value
#             except:
#                 pass
            try:
                if value is not None and keep_func(value):
                    merged[i].append(value)
            except KeyError:
                merged[i] = [value]
    for key in merged.keys():
        try:
            merged[key] = merge_func(merged[key])
        except:
            merged[key] = merged[key][0]
    return [(key, new_column_id, value) for key, value in merged.items()]

# map itemids to column numbers
def get_column_set(itemids, map_itemid_index):
    return set([map_itemid_index[itemid][0] for itemid in itemids])

# test on Glasgow coma scale
adm = np.load(os.path.join(SOURCEDIR, 'adm-194627.npy')).tolist()
print(merge_items(adm['timeseries']['codes'], set([23634]), 123, keep_func=lambda x: True))
# print(get_column_set([454,223900], map_itemid_index))

[(0, 123, -5220.0), (1, 123, -4620.0), (2, 123, -4380.0), (3, 123, -3120.0), (4, 123, 60.0), (5, 123, 5640.0), (6, 123, 11280.0), (7, 123, 11340.0), (8, 123, 12600.0), (9, 123, 12780.0), (10, 123, 12900.0), (11, 123, 13080.0), (12, 123, 13920.0), (13, 123, 13980.0), (14, 123, 14040.0), (15, 123, 14100.0), (16, 123, 14160.0), (17, 123, 14280.0), (18, 123, 14340.0), (19, 123, 14880.0), (20, 123, 14940.0), (21, 123, 15000.0), (22, 123, 15060.0), (23, 123, 15300.0), (24, 123, 15360.0), (25, 123, 15720.0), (26, 123, 15780.0), (27, 123, 16140.0), (28, 123, 16200.0), (29, 123, 16260.0), (30, 123, 16320.0), (31, 123, 16440.0), (32, 123, 16500.0), (33, 123, 16560.0), (34, 123, 16620.0), (35, 123, 16740.0), (36, 123, 17100.0), (37, 123, 17160.0), (38, 123, 17280.0), (39, 123, 17340.0), (40, 123, 17460.0), (41, 123, 17520.0), (42, 123, 18000.0), (43, 123, 18240.0), (44, 123, 18360.0), (45, 123, 18420.0), (46, 123, 18480.0), (47, 123, 18720.0), (48, 123, 18780.0), (49, 123, 18840.0), (50, 123, 193

## Making the map of features and itemids

Here we define the map between features and itemids. Most features come from two data sources and we have to manually define the relationship between features and itemids and merge the data. We also assign the rank of column for each feature.

In [4]:
# derive 17 features from manually selected itemids
# https://docs.google.com/spreadsheets/d/1e2KqLn3LTvcUwpSe5oE2ADwIEmUH9Xh54VADYVQ9mEQ/edit?ts=5960262a#gid=750248768
feature_itemids = OrderedDict({
    'gcsverbal': [723, 223900],
    'gcsmotor': [454, 223901],
    'gcseyes': [184, 220739],
#     'glasgow_coma_scale': [454, 223900],
#     'systolic_blood_pressure_abp_high_6': [6, 220050],
#     'systolic_blood_pressure_abp_high_51': [51, 220050],
#     'systolic_blood_pressure_abp_high_6701': [6701, 220050],
    'systolic_blood_pressure_abp_mean': [51, 442, 455, 6701, 220050, 220179],
#     'systolic_blood_pressure_abp_high_mean': [6, 51, 6701, 220050],
#     'systolic_blood_pressure_abp_high_max': [6, 51, 6701, 220050],
#     'systolic_blood_pressure_abp_high_min': [6, 51, 6701, 220050],
#     'systolic_blood_pressure_abp_low': [6],
#     'systolic_blood_pressure_nbp_high': [455, 220179],
#     'systolic_blood_pressure_nbp_low': [],
    'heart_rate': [211, 220045],
    'body_temperature': {
        'f': [678, 223761],
        'c': [676, 223762]
    },
    'pao2': [50821],
    'fio2': [50816, 223835, 3420, 3422, 190],
#     'pao2_fio2_ratio': [50821, 50816],
    'urinary_output': [40055,
43175,
40069,
40094,
40715,
40473,
40085,
40057,
40056,
40405,
40428,
40086,
40096,
40651,
226559,
226560,
226561,
226584,
226563,
226564,
226565,
226567,
226557,
226558,
227488,
227489],
    'serum_urea_nitrogen_level': [51006],
#     'white_blood_cells_count_51300': [51300],
#     'white_blood_cells_count_51301': [51301],
    'white_blood_cells_count_mean': [51300, 51301],
#     'white_blood_cells_count_max': [51300, 51301],
#     'white_blood_cells_count_min': [51300, 51301],
#     'serum_bicarbonate_level_50803': [50803],
#     'serum_bicarbonate_level_50804': [50804],
#     'serum_bicarbonate_level_50802': [50802],
    'serum_bicarbonate_level_mean': [50882],
#     'serum_bicarbonate_level_max': [50803, 50804, 50802],
#     'serum_bicarbonate_level_min': [50803, 50804, 50802],
#     'sodium_level_50824': [50824],
#     'sodium_level_50983': [50983],
    'sodium_level_mean': [50824, 50983],
#     'sodium_level_max': [50824, 50983],
#     'sodium_level_min': [50824, 50983],
#     'potassium_level_50822': [50822],
#     'potassium_level_50971': [50971],
    'potassium_level_mean': [50822, 50971],
#     'potassium_level_max': [50822, 50971],
#     'potassium_level_min': [50822, 50971],
    'bilirubin_level': [50885],
#     'type_of_admission': [],
#     'acquired_immunodeficiency_syndrome': [],
#     'metastatic_cancer': [],
#     'hematologic_malignancy': []
    'timestamp': [],
    'aid': []
})

merge_funcs = {
    'mean': np.mean,
    'max': np.max,
    'min': np.min
}

map_feature_colids = {}
t = 0
for key in feature_itemids.keys():
    map_feature_colids[key] = t
    t += 1
map_feature_colids
print(len(map_feature_colids))
print(map_feature_colids)

17
{'gcsverbal': 0, 'gcsmotor': 1, 'gcseyes': 2, 'systolic_blood_pressure_abp_mean': 3, 'heart_rate': 4, 'body_temperature': 5, 'pao2': 6, 'fio2': 7, 'urinary_output': 8, 'serum_urea_nitrogen_level': 9, 'white_blood_cells_count_mean': 10, 'serum_bicarbonate_level_mean': 11, 'sodium_level_mean': 12, 'potassium_level_mean': 13, 'bilirubin_level': 14, 'timestamp': 15, 'aid': 16}


In [5]:
np.save(os.path.join(RAWDIR, 'map_feature_colids.npy'), map_feature_colids)

## Collect names of columns for verification

In [6]:
conn = getConnection()
cur = conn.cursor()
for feature, itemids in feature_itemids.items():
    if len(itemids) == 0:
        continue
    if type(itemids) == type({}):
        for key, value in itemids.items():
            sql = 'select itemid, label from mimiciii.d_items where itemid in ({0}) union all select itemid, label from mimiciii.d_labitems where itemid in ({0})'.format(','.join(list(map(str, value))))
            cur.execute(sql)
            res = cur.fetchall()
            print(feature + ' ' + key)
            for r in res:
                print('{0},{1}'.format(r[0], r[1]))
            print()
    else:  
        sql = 'select itemid, label from mimiciii.d_items where itemid in ({0}) union all select itemid, label from mimiciii.d_labitems where itemid in ({0})'.format(','.join(list(map(str, itemids))))
        cur.execute(sql)
        res = cur.fetchall()
        print(feature)
        for r in res:
            print('{0},{1}'.format(r[0], r[1]))
        print()

gcsverbal
723,Verbal Response
223900,GCS - Verbal Response

gcsmotor
454,Motor Response
223901,GCS - Motor Response

gcseyes
184,Eye Opening
220739,GCS - Eye Opening

systolic_blood_pressure_abp_mean
51,Arterial BP [Systolic]
442,Manual BP [Systolic]
455,NBP [Systolic]
6701,Arterial BP #2 [Systolic]
220050,Arterial Blood Pressure systolic
220179,Non Invasive Blood Pressure systolic

heart_rate
211,Heart Rate
220045,Heart Rate

body_temperature f
678,Temperature F
223761,Temperature Fahrenheit

body_temperature c
676,Temperature C
223762,Temperature Celsius

pao2
50821,pO2

fio2
190,FiO2 Set
3420,FIO2
3422,FIO2 [Meas]
223835,Inspired O2 Fraction
50816,Oxygen

urinary_output
40055,Urine Out Foley
40056,Urine Out Lt Nephrostomy
40057,Urine Out Rt Nephrostomy
40069,Urine Out Void
40085,Urine Out Incontinent
40086,Drain Out #2 Pigtail
40094,Urine Out Condom Cath
40096,Urine Out Ureteral Stent #1
40405,Urine Out Other
40428,Urine Out Straight Cath
40473,Urine Out IleoConduit
40651,Urine Out 

## Extract temporal features

Since the number of temporal features is limited, we manually define the processing method for each feature in the following code.

- body temperature: convert Farenheit to Celcius, use Celcius in priority in confliction
- urinary output: use the sum of all related itemids
- other features: use mean value when meeting confliction

In [7]:
def extract_serial_features(aid, res):
    timeseries, general, icd9 = res['timeseries']['codes'], res['general'], res['icd9']
    D = res['timeseries']['features']
    assert general[0] == aid
    new_timeseries = []
    
    # body temperature
    ctemp = merge_items(
        timeseries, 
        get_column_set(feature_itemids['body_temperature']['c'], map_itemid_index), 
        map_feature_colids['body_temperature']
    )
    ctempids = set([x[0] for x in ctemp])
    
    ftemp = merge_items(
        timeseries, 
        get_column_set(feature_itemids['body_temperature']['f'], map_itemid_index), 
        map_feature_colids['body_temperature']
    )
    ftemp = [(x[0], x[1], (x[2] - 32) * 5 / 9.0) for x in ftemp if x[0] not in ctempids]
    ftemp = [x for x in ftemp if x[2] > 0]
    new_timeseries.append(ctemp + ftemp)
    
    # urinary output
    new_timeseries.append(merge_items(
        timeseries, 
        get_column_set(feature_itemids['urinary_output'], map_itemid_index), 
        map_feature_colids['urinary_output'],
        merge_func=np.sum
    ))
    
    # serum urea nitrogen/white blood cells count/serum bicarbonate/sodium/potassium/bilirubin level, just merge
    for itemname in ['gcsverbal',
                     'gcsmotor',
                     'gcseyes',
                    'systolic_blood_pressure_abp_mean',
                    'heart_rate',
                    'pao2',
                    'fio2',
                    'serum_urea_nitrogen_level',
                    'white_blood_cells_count',
                    'serum_bicarbonate_level',
                    'sodium_level',
                    'potassium_level',
                    'bilirubin_level']:
        tks = [tk for tk in feature_itemids.keys() if tk.startswith(itemname)]
        for tk in tks:
            try:
                merge_func = merge_funcs[tk.split('_')[-1]]
            except:
                merge_func = np.mean
            new_timeseries.append(merge_items(
                timeseries, 
                get_column_set(feature_itemids[tk], map_itemid_index), 
                map_feature_colids[tk],
                merge_func=merge_func
            ))
            
    # get timestamps
    valid_timestamps = list(itertools.chain(*[[t[0] for t in tt] for tt in new_timeseries]))
    max_timestamp = max(valid_timestamps)
    timestamps = [(t[0], map_feature_colids['timestamp'], t[2]) for t in timeseries if t[0] <= max_timestamp and t[1] == D - 2]
    new_timeseries.append(timestamps)
    timelength = max([t[2] for t in timestamps]) - min([t[2] for t in timestamps])
    
    # aid
    aids = [(t[0], map_feature_colids['aid'], t[2]) for t in timeseries if t[0] <= max_timestamp and t[1] == D - 1]
    new_timeseries.append(aids)
    
    res2 = list(itertools.chain(*new_timeseries))
    return {'timestep':max_timestamp + 1,'features':len(map_feature_colids),'codes':res2, 'timelength':timelength}

adm = np.load(os.path.join('admdata', 'adm-127294.npy')).tolist()
extract_serial_features(127294, adm)

{'codes': [(5, 5, 36.944444444444443),
  (19, 5, 37.611111111111114),
  (34, 5, 36.777777777777779),
  (42, 5, 36.5),
  (57, 5, 36.333333333333336),
  (60, 5, 36.833333333333336),
  (65, 5, 37.166666666666664),
  (77, 5, 37.333333333333336),
  (95, 5, 37.5),
  (107, 5, 37.333333333333336),
  (129, 5, 37.0),
  (158, 5, 37.444444444444443),
  (159, 5, 37.666666666666664),
  (177, 5, 37.0),
  (187, 5, 37.944444444444443),
  (200, 5, 37.833333333333336),
  (201, 5, 37.666666666666664),
  (204, 5, 37.777777777777779),
  (205, 5, 37.722222222222221),
  (209, 5, 37.722222222222221),
  (222, 5, 38.333333333333336),
  (225, 5, 38.166666666666664),
  (228, 5, 38.111111111111114),
  (230, 5, 38.111111111111114),
  (239, 5, 38.0),
  (242, 5, 37.555555555555557),
  (243, 5, 37.555555555555557),
  (247, 5, 37.555555555555557),
  (258, 5, 36.888888888888886),
  (271, 5, 37.111111111111114),
  (279, 5, 37.5),
  (291, 5, 37.388888888888886),
  (304, 5, 37.611111111111114),
  (318, 5, 37.444444444444443

## Extract non-temporal features

In [8]:
def extract_adm_features(processed_adm):
    # get from previous in database
    return processed_adm['adm_features']

adm = np.load(os.path.join(PROCESSED_DB_DIR, 'adm-107051.npy')).tolist()
extract_adm_features(adm)

(29847.576240000002, 0, 0, 0, 2)

## Extract mortality labels

In [10]:
def extract_adm_labels(aid):
    admlabel = np.load(os.path.join(LABELDIR, 'adm-%.6d.npy' % aid)).tolist()
    adm_labels = (
        admlabel['mor'],
        admlabel['mor24'],
        admlabel['mor48'],
        admlabel['mor72'],
        admlabel['mor30d'],
        admlabel['mor1y'],
    )
    return adm_labels

extract_adm_labels(107051)

(0, 0, 0, 0, 0, 0)

## Store raw data without filtering on the time length of record, and get some stats on it

In [11]:
def extract_adm(aid):
    admres = np.load(os.path.join(SOURCEDIR, 'adm-{0}.npy'.format(str('%.6d' % aid)))).tolist()
    processed_admres = np.load(os.path.join(PROCESSED_DB_DIR, 'adm-{0}.npy'.format(str('%.6d' % aid)))).tolist()
    res = {
        'serial_features': extract_serial_features(aid, admres),
        'adm_features': extract_adm_features(processed_admres),
        'adm_labels': extract_adm_labels(aid)
    }
    np.save(os.path.join(RAWDIR, 'adm-{0}.npy'.format(str('%.6d' % aid))), res)
    print('finished {0}!'.format(aid))
    
extract_adm(164444)

finished 164444!


In [12]:
p = Pool()
for aid in valid_aids:
    p.apply_async(extract_adm, args=(aid,))
p.close()
p.join()

finished 100001!
finished 100016!
finished 100007!
finished 100006!
finished 100009!
finished 100028!
finished 100017!
finished 100030!
finished 100010!
finished 100021!
finished 100024!
finished 100040!
finished 100012!
finished 100034!
finished 100033!
finished 100011!
finished 100041!
finished 100038!
finished 100031!
finished 100036!
finished 100050!
finished 100046!
finished 100066!
finished 100059!
finished 100035!
finished 100071!
finished 100065!
finished 100063!
finished 100075!
finished 100072!
finished 100053!
finished 100045!
finished 100078!
finished 100037!
finished 100087!
finished 100099!
finished 100074!
finished 100113!
finished 100094!
finished 100058!
finished 100112!
finished 100095!
finished 100085!
finished 100109!
finished 100114!
finished 100116!
finished 100140!
finished 100119!
finished 100118!
finished 100126!
finished 100124!
finished 100131!
finished 100108!
finished 100136!
finished 100142!
finished 100147!
finished 100143!
finished 100146!
finished 10016

finished 101271!
finished 101237!
finished 101215!
finished 101272!
finished 101289!
finished 101277!
finished 101274!
finished 101278!
finished 101306!
finished 101311!
finished 101297!
finished 101315!
finished 101280!
finished 101263!
finished 101286!
finished 101324!
finished 101276!
finished 101310!
finished 101317!
finished 101295!
finished 101241!
finished 101291!
finished 101303!
finished 101348!
finished 101325!
finished 101326!
finished 101343!
finished 101329!
finished 101331!
finished 101340!
finished 101333!
finished 101296!
finished 101353!
finished 101341!
finished 101335!
finished 101360!
finished 101334!
finished 101299!
finished 101374!
finished 101302!
finished 101371!
finished 101352!
finished 101394!
finished 101381!
finished 101373!
finished 101344!
finished 101399!
finished 101400!
finished 101361!
finished 101424!
finished 101388!
finished 101379!
finished 101350!
finished 101397!
finished 101404!
finished 101363!
finished 101430!
finished 101432!
finished 10132

finished 102480!
finished 102566!
finished 102524!
finished 102563!
finished 102515!
finished 102567!
finished 102595!
finished 102572!
finished 102535!
finished 102576!
finished 102565!
finished 102557!
finished 102513!
finished 102438!
finished 102603!
finished 102586!
finished 102599!
finished 102612!
finished 102658!
finished 102641!
finished 102651!
finished 102629!
finished 102625!
finished 102587!
finished 102646!
finished 102589!
finished 102644!
finished 102622!
finished 102602!
finished 102600!
finished 102673!
finished 102605!
finished 102636!
finished 102616!
finished 102613!
finished 102649!
finished 102657!
finished 102497!
finished 102672!
finished 102610!
finished 102666!
finished 102624!
finished 102664!
finished 102676!
finished 102588!
finished 102678!
finished 102645!
finished 102693!
finished 102713!
finished 102684!
finished 102710!
finished 102698!
finished 102716!
finished 102696!
finished 102682!
finished 102722!
finished 102683!
finished 102739!
finished 10274

finished 103781!
finished 103788!
finished 103843!
finished 103841!
finished 103837!
finished 103794!
finished 103836!
finished 103878!
finished 103859!
finished 103741!
finished 103867!
finished 103829!
finished 103853!
finished 103879!
finished 103884!
finished 103894!
finished 103854!
finished 103871!
finished 103873!
finished 103754!
finished 103903!
finished 103815!
finished 103890!
finished 103848!
finished 103869!
finished 103920!
finished 103907!
finished 103892!
finished 103883!
finished 103882!
finished 103900!
finished 103863!
finished 103919!
finished 103880!
finished 103886!
finished 103901!
finished 103856!
finished 103941!
finished 103906!
finished 103915!
finished 103889!
finished 103942!
finished 103922!
finished 103937!
finished 103966!
finished 103972!
finished 103973!
finished 103988!
finished 103933!
finished 103928!
finished 103925!
finished 103998!
finished 103949!
finished 103943!
finished 103954!
finished 103921!
finished 104004!
finished 103932!
finished 10401

finished 105173!
finished 105041!
finished 105136!
finished 105148!
finished 104984!
finished 105106!
finished 105153!
finished 105011!
finished 105152!
finished 105157!
finished 105146!
finished 105206!
finished 105158!
finished 105193!
finished 105167!
finished 105163!
finished 105058!
finished 105094!
finished 105141!
finished 105080!
finished 105196!
finished 105169!
finished 105123!
finished 105194!
finished 105218!
finished 105222!
finished 105110!
finished 105187!
finished 105216!
finished 105159!
finished 105235!
finished 105239!
finished 105264!
finished 105091!
finished 105240!
finished 105175!
finished 105247!
finished 105252!
finished 105164!
finished 105202!
finished 105254!
finished 105213!
finished 105208!
finished 105229!
finished 105217!
finished 105267!
finished 105288!
finished 105287!
finished 105268!
finished 105282!
finished 105262!
finished 105316!
finished 105276!
finished 105285!
finished 105237!
finished 105302!
finished 105317!
finished 105244!
finished 10528

...

finished 194293!
finished 194201!
finished 194266!
finished 194263!
finished 194226!
finished 194132!
finished 194192!
finished 194043!
finished 194204!
finished 194117!
finished 194227!
finished 194229!
finished 194275!
finished 194237!
finished 194210!
finished 194319!
finished 194324!
finished 194273!
finished 194311!
finished 194315!
finished 194252!
finished 194289!
finished 194278!
finished 194233!
finished 194335!
finished 194254!
finished 194291!
finished 194253!
finished 194309!
finished 194265!
finished 194222!
finished 194086!
finished 194242!
finished 194301!
finished 194340!
finished 194316!
finished 194341!
finished 194332!
finished 194328!
finished 194356!
finished 194280!
finished 194359!
finished 194346!
finished 194322!
finished 194360!
finished 194355!
finished 194362!
finished 194376!
finished 194325!
finished 194353!
finished 194292!
finished 194348!
finished 194375!
finished 194333!
finished 194380!
finished 194082!
finished 194269!
finished 194386!
finished 19436

finished 195466!
finished 195480!
finished 195465!
finished 195492!
finished 195485!
finished 195535!
finished 195508!
finished 195471!
finished 195509!
finished 195442!
finished 195472!
finished 195499!
finished 195528!
finished 195503!
finished 195536!
finished 195525!
finished 195461!
finished 195473!
finished 195510!
finished 195547!
finished 195551!
finished 195554!
finished 195502!
finished 195552!
finished 195527!
finished 195344!
finished 195522!
finished 195451!
finished 195438!
finished 195560!
finished 195537!
finished 195501!
finished 195559!
finished 195550!
finished 195595!
finished 195458!
finished 195384!
finished 195600!
finished 195563!
finished 195610!
finished 195612!
finished 195555!
finished 195574!
finished 195598!
finished 195619!
finished 195581!
finished 195605!
finished 195608!
finished 195487!
finished 195556!
finished 195623!
finished 195631!
finished 195604!
finished 195627!
finished 195614!
finished 195583!
finished 195569!
finished 195642!
finished 19563

finished 196823!
finished 196809!
finished 196828!
finished 196841!
finished 196831!
finished 196854!
finished 196778!
finished 196707!
finished 196829!
finished 196849!
finished 196839!
finished 196847!
finished 196861!
finished 196864!
finished 196850!
finished 196870!
finished 196803!
finished 196860!
finished 196881!
finished 196889!
finished 196893!
finished 196871!
finished 196822!
finished 196884!
finished 196856!
finished 196844!
finished 196859!
finished 196898!
finished 196792!
finished 196879!
finished 196896!
finished 196649!
finished 196867!
finished 196907!
finished 196705!
finished 196926!
finished 196899!
finished 196851!
finished 196927!
finished 196906!
finished 196868!
finished 196916!
finished 196936!
finished 196920!
finished 196914!
finished 196922!
finished 196947!
finished 196958!
finished 196887!
finished 196959!
finished 196952!
finished 196745!
finished 196921!
finished 196930!
finished 196855!
finished 196883!
finished 196948!
finished 196977!
finished 19696

finished 198028!
finished 198062!
finished 198015!
finished 198085!
finished 198019!
finished 198046!
finished 198091!
finished 198090!
finished 198022!
finished 198100!
finished 198129!
finished 197965!
finished 198072!
finished 198122!
finished 198103!
finished 198087!
finished 197990!
finished 198146!
finished 198166!
finished 198127!
finished 198144!
finished 198070!
finished 198182!
finished 198082!
finished 198163!
finished 197813!
finished 198180!
finished 198171!
finished 198207!
finished 198187!
finished 198183!
finished 198190!
finished 198152!
finished 198020!
finished 198208!
finished 198201!
finished 198231!
finished 198238!
finished 198156!
finished 198142!
finished 198124!
finished 198220!
finished 198098!
finished 198222!
finished 198125!
finished 198184!
finished 198249!
finished 198168!
finished 198262!
finished 198225!
finished 198259!
finished 198219!
finished 198248!
finished 198212!
finished 198205!
finished 198252!
finished 198266!
finished 198283!
finished 19825

finished 199515!
finished 199526!
finished 199394!
finished 199540!
finished 199495!
finished 199565!
finished 199280!
finished 199560!
finished 199511!
finished 199524!
finished 199488!
finished 199546!
finished 199535!
finished 199586!
finished 199566!
finished 199579!
finished 199382!
finished 199582!
finished 199585!
finished 199572!
finished 199597!
finished 199593!
finished 199590!
finished 199505!
finished 199545!
finished 199533!
finished 199584!
finished 199626!
finished 199573!
finished 199642!
finished 199568!
finished 199598!
finished 199637!
finished 199522!
finished 199571!
finished 199608!
finished 199631!
finished 199623!
finished 199610!
finished 199647!
finished 199577!
finished 199558!
finished 199636!
finished 199675!
finished 199669!
finished 199574!
finished 199678!
finished 199654!
finished 199662!
finished 199581!
finished 199632!
finished 199580!
finished 199543!
finished 199604!
finished 199645!
finished 199641!
finished 199562!
finished 199643!
finished 19954

In [ ]:
# # some check and stats
# def check_aid(aid):
#     res = np.load(os.path.join(RAWDIR, 'adm-{0}.npy'.format(str('%.6d' % aid)))).tolist()
#     mortal, die24, die24_48, die48_72, die30days, die1year = res['adm_labels']
#     admfs = res['adm_features']
#     admlb = res['adm_labels']
#     try:
#         assert (((~(die24 | die24_48 | die48_72)) | die30days) & 1) == 1
#         assert (((~(die24 | die24_48 | die48_72 | die30days)) | die1year) & 1) == 1
#     except:
#         print(admlb)
#     return admfs, admlb, aid

# admission_ids_17f_raw = [re.match(r'adm\-(\d+)\.npy', x) for x in os.listdir(RAWDIR)]
# admission_ids_17f_raw = [int(x.group(1)) for x in admission_ids_17f_raw if x is not None]

# p = Pool()
# ress = p.map(check_aid, admission_ids_17f_raw)
# ress = list(ress)

# print('mortal:', np.sum([x[1][0] for x in ress]))
# print('24hrs:', np.sum([x[1][1] for x in ress]))
# print('24~48hrs:', np.sum([x[1][2] for x in ress]))
# print('48~72hrs:', np.sum([x[1][3] for x in ress]))
# print('<=30d:', np.sum([x[1][4] for x in ress]))
# print('<=1yr:', np.sum([x[1][5] for x in ress]))

# # age, type_of_admission, aids, metastatic_cancer, hematologic_malignancy
# print('aids:', np.sum([x[0][2] for x in ress]))
# print('metastatic_cancer:', np.sum([x[0][3] for x in ress]))
# print('hematologic_malignancy:', np.sum([x[0][4] for x in ress]))

# from collections import Counter

# toc_counter = Counter([x[0][1] for x in ress])
# plt.bar([0,1,2,3], [value for _, value in toc_counter.items()], 0.5)
# plt.xticks(np.arange(4), [rev_mapping['admission_type_ms'][x] for x, _ in toc_counter.items() ])

# x2 = [x[0][0]/365.25 for x in ress]
# np.max(x2)

# plt.hist([x[0][0]/365 for x in ress], 40)

## Select admissions with > xxhrs records

In this step we only keep admissions with record length > 24/48 hrs.

In [ ]:
def extract_time_series(aidres):
    time_series = aidres['serial_features']
    tn = time_series['timestep']
    fn = time_series['features']
    codes = time_series['codes']
    series = [[None for ft in range(fn)] for tt in range(tn)]
    for tt, ft, value in codes:
        try:
            series[tt][ft] = value
        except:
            print(tt, ft, value, tn, fn)
    return series

def check_adm_hrs_pass(admres, hrs):
    return admres['serial_features']['timelength'] > hrs * 3600.0
    
def extract_data(aid, hrs):
    admres = np.load(os.path.join(RAWDIR, 'adm-%.6d.npy' % aid)).tolist()
    if check_adm_hrs_pass(admres, hrs):
        ori_admres = np.load(os.path.join(SOURCEDIR, 'adm-%.6d.npy' % aid)).tolist()
        return (
            extract_time_series(admres),
            ori_admres['icd9'],
            ori_admres['general'],
            admres['adm_features'],
            admres['adm_labels']
        )
    else:
        return None
        
def collect_admissions_with_more_than_hrs(hrs):
#     raw_aids = [re.match(r'adm\-(\d+)\.npy', x) for x in os.listdir(RAWDIR)]
#     raw_aids = [int(x.group(1)) for x in raw_aids if x is not None]
    
    processed_data_all = np.load(os.path.join(TARGETDIR, '%dhrs' % hrs, 'DB_merged_%dhrs.npy' % hrs)).tolist()
    processed_valid_aids = sorted([t[0][-1] for t in processed_data_all])
    
    HRDIR = os.path.join(TARGETDIR, '%dhrs_raw' % hrs)
    if not os.path.exists(HRDIR):
        os.makedirs(HRDIR)
        
    p = Pool()
    collec = [p.apply_async(extract_data, args=(aid, hrs)) for aid in processed_valid_aids]
    p.close()
    p.join()
    collec = [x.get() for x in collec]
    collec = [x for x in collec if x is not None]
    
    data_all = [r[0] for r in collec]
    label_icd9_all = [r[1] for r in collec]
#     label_mor_all = [r[2][:6] for r in collec]
    label_mor_all = [r[2] for r in collec]
    adm_features_all = [r[3] for r in collec]
    adm_labels_all = [r[4] for r in collec]
    
    np.save(os.path.join(HRDIR, 'DB_merged_%dhrs.npy' % hrs), data_all)
    np.save(os.path.join(HRDIR, 'ICD9-%dhrs.npy' % hrs), label_icd9_all)
    np.save(os.path.join(HRDIR, 'AGE_LOS_MORTALITY_%dhrs.npy' % hrs), label_mor_all)
    np.save(os.path.join(HRDIR, 'ADM_FEATURES_%dhrs.npy' % hrs), adm_features_all)
    np.save(os.path.join(HRDIR, 'ADM_LABELS_%dhrs.npy' % hrs), adm_labels_all)

# > 24hrs
collect_admissions_with_more_than_hrs(24)

In [ ]:
collect_admissions_with_more_than_hrs(48)

In [ ]:
admres = np.load(os.path.join(RAWDIR, 'adm-%.6d.npy' % 199999)).tolist()
admres['serial_features']['codes']